In [16]:
import pandas as pd
import math
import numpy as np
import sqlite3
from werkzeug.security import generate_password_hash

In [17]:
tahun = 2021
xl = pd.ExcelFile("data/dummies.xlsx")

In [18]:
xl.sheet_names

['user', 'profil', 'penilaian', 'kriteria', 'skor']

In [19]:
user = xl.parse(xl.sheet_names[0])
user['password'] = user.apply(lambda x: generate_password_hash(x['password'], method='sha256'), axis=1)
user = user.rename(columns={'uid':'id'})
user.head()

,username,id,password
0,kepsek,u001,sha256$91hoqmxvZlNwDIx2$06f47f2fea6fb11aaed24a...
1,admin,u004,sha256$xuHVMRvdp6rh5aZF$70bbf97c25adea024a0217...


In [20]:
profil = xl.parse(xl.sheet_names[1])
profil['nuptk'] = profil['nuptk'].fillna(0)
profil['nuptk'] = profil['nuptk'].astype(int).astype(str)
profil['nrg'] = profil['nrg'].fillna(0)
profil['nrg'] = profil['nrg'].astype(int).astype(str)
profil['ttl'] = pd.to_datetime(profil['ttl'])
profil['tmt'] = pd.to_datetime(profil['tmt'])
profil['mulai'] = pd.to_datetime(profil['mulai'])
profil.head()

,uid,nip,karpeg,nuptk,nrg,nama,ttl,gol,tmt,mulai,pendidikan,gender,mata_pelajaran,bidang_yang_diampu
0,u001,198505192015032856,B05017957,2851763664130020,141561293313,"Jasmin, S.Pd.SD",1974-11-30,VIA,1998-03-05,2006-10-15,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Kepala Sekolah
1,u002,198505192015031819,B05017869,2851763664130256,141561293502,"Kutsiyah, S.Pd",1984-04-29,IIID,2007-03-13,2012-01-18,Pendidikan Jasmani,Perempuan,Pendidikan Jasmani dan Kesehatan,Guru Pendidikan Jasmani dan Kesehatan
2,u003,198505192015031387,B05017826,2851763664130231,141561293323,"Sendja Indah, S.Pd",1984-01-28,IIID,2006-09-01,2012-11-05,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas I A dan B
3,u004,198505192015031907,B05017856,0,0,Nama52,1988-10-03,IIIC,2011-08-22,2013-01-26,NaN,Laki-laki,Admin,NaN
4,u005,198505192015032861,B05017963,2851763664130246,141561293388,"Lestari, S.Pd",1987-08-12,IIIC,2011-05-07,2013-11-24,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas III A


In [21]:
nilai = xl.parse(xl.sheet_names[2])
nilai.head()

,pid,uid,tgl_penilaian,tahun,A01,A02,A03,A04,A05,A06,...,L03,M01,M02,M03,N01,N02,N03,N04,N05,N06
0,p001,u002,2020-12-21,2020,2,0,0,1,2,1,...,1,0,1,0,0,1,1,2,0,1
1,p002,u003,2020-12-21,2020,2,1,2,2,1,1,...,1,1,1,2,2,1,2,0,2,1
2,p003,u005,2020-12-21,2020,2,1,2,1,1,0,...,2,2,2,1,1,1,1,2,2,1
3,p004,u006,2020-12-21,2020,1,1,2,1,2,2,...,2,1,1,2,1,2,1,1,2,2
4,p005,u007,2020-12-21,2020,1,2,0,0,0,1,...,1,1,2,0,1,2,1,1,1,0


In [22]:
kriteria = xl.parse(xl.sheet_names[3])
kriteria.head()

,indikator_id,kompetensi,indikator
0,A01,Mengenal karakteristik peserta didik,Guru dapat mengidentifikasi karakteristik bela...
1,A02,Mengenal karakteristik peserta didik,Guru memastikan bahwa semua peserta didik mend...
2,A03,Mengenal karakteristik peserta didik,Guru dapat mengatur kelas untuk memberikan kes...
3,A04,Mengenal karakteristik peserta didik,Guru mencoba mengetahui penyebab penyimpangan ...
4,A05,Mengenal karakteristik peserta didik,Guru membantu mengembangkan potensi dan mengat...


In [23]:
skor = xl.parse(xl.sheet_names[4])
skor.head()

,skor,keterangan
0,0,Tidak ada bukti (Tidak terpenuhi)
1,1,Terpenuhi sebagian
2,2,Seluruhnya terpenuhi


In [24]:
con = sqlite3.connect("db/data.db",check_same_thread=False)
user.to_sql('user',con=con,if_exists='replace',index=False)
profil.to_sql('profil',con=con,if_exists='replace',index=False)
nilai.to_sql('nilai',con=con,if_exists='replace',index=False)
kriteria.to_sql('kriteria',con=con,if_exists='replace',index=False)
skor.to_sql('skor',con=con,if_exists='replace',index=False)

In [25]:
def normalization(matrix):
    # Transpose Decision Matrix
    matrix = matrix.transpose()
    row_values = []
    norm_matrix = []
    
    for i in range(matrix.shape[0]): # Looping per baris (kriteria)
        # Menghitung sum tiap x_{ij}^2
        sum_row = sum([pow(x,2) for x in matrix[i]])
        
        for j in range(matrix[i].shape[0]): # Looping per kolom (alternatif)
            # membangi nilai asli x_{ij} dengan hasil akar
            r_value = matrix[i][j] / math.sqrt(sum_row)
            
            # Masukkan hasil normalisasi ke list tiap baris
            row_values.append(r_value)
        
        #Masukkan hasil normalisasi per baris ke matrix normalisasi
        norm_matrix.append(row_values)
        
        #Kosongkan list normalisasi perbaris
        row_values = []
            
    # Ubah dalam bentuk numpy array
    norm_matrix = np.asarray(norm_matrix)
    
    # Return dalam bentuk transporse agar kembali ke format awal
    return norm_matrix.transpose()

In [26]:
# Implementasi Menghitung Nilai 
def optimize_value(w_matrix):
    y_values = []
    
    for i in range(w_matrix.shape[0]):
        max_val = []
        min_val = []
        
        for j in range(w_matrix[i].shape[0]):
            max_val.append(w_matrix[i][j])
        
        y = sum(max_val) - sum(min_val)
        y_values.append(y)
    
    return np.asarray(y_values)

In [27]:
def bobot(nilai):
    if nilai <=25:
        return 1
    elif nilai >25 and nilai <=50:
        return 2
    elif nilai >50 and nilai <=75:
        return 3
    else:
        return 4

In [28]:
def moora(tahun):
    nilai = pd.read_sql_query(
        f"select * from nilai where tahun = {tahun}",
        con=con
    )

    kriteria = pd.read_sql_query(
        "select * from kriteria",
        con=con
    )

    profil = pd.read_sql_query(
        "select * from profil",
        con=con
    )

    df = nilai.iloc[:,:4].copy()
    for m in  kriteria['indikator_id'].str[0].unique():
        df[m] = pd.DataFrame(nilai[[n for n in nilai.columns if n.startswith("A")]].sum(axis=1)*100/(len([n for n in nilai.columns if n.startswith("A")])*2), columns=['A']).apply(lambda x: bobot(x.values[0]), axis=1)
    return df.iloc[:,:4].join(pd.DataFrame(optimize_value(normalization(df.iloc[:,4:].values)),columns=['skor'])).sort_values(by='skor',ascending=False).set_index('uid').join(profil.set_index('uid')).reset_index(), \
        df.rename(columns={n:kriteria[kriteria['indikator_id'].str[0]==n]['kompetensi'].unique()[0] for n in kriteria['indikator_id'].str[0].unique()})

In [29]:
df,dx = moora(tahun)
dx

,pid,uid,tgl_penilaian,tahun,Mengenal karakteristik peserta didik,Menguasai teori belajar dan prinsip‐prinsip pembelajaran yang mendidik,Pengembangan Kurikulum,Kegiatan Pembelajaran yang Mendidik,Memahami dan mengembangkan potensi,Komunikasi dengan peserta didik,Penilaian dan evaluasi,"Bertindak sesuai dengan norma agama, hukum, sosial dan kebudayaan nasional Indonesia",Menunjukkan pribadi yang dewasa dan teladan,"Etos kerja, tanggung jawab yang tinggi, dan rasa bangga menjadi guru","Bersikap inklusif, bertindak objektif, serta tidak Diskriminatif","Komunikasi dengan sesama guru, tenaga kependidikan, orang tua peserta didik, dan masyarakat","Penguasaan materi, struktur, konsep, dan pola pikir keilmuan yang mendukung mata pelajaran yang diampu",Mengembangkan keprofesionalan melalui tindakan reflektif
0,p051,u002,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,p052,u003,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,p053,u005,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3,p054,u006,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,p055,u007,2021-12-21 00:00:00,2021,2,2,2,2,2,2,2,2,2,2,2,2,2,2
5,p056,u008,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
6,p057,u009,2021-12-21 00:00:00,2021,2,2,2,2,2,2,2,2,2,2,2,2,2,2
7,p058,u010,2021-12-21 00:00:00,2021,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8,p059,u011,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3
9,p060,u012,2021-12-21 00:00:00,2021,3,3,3,3,3,3,3,3,3,3,3,3,3,3


In [30]:
df

,uid,pid,tgl_penilaian,tahun,skor,nip,karpeg,nuptk,nrg,nama,ttl,gol,tmt,mulai,pendidikan,gender,mata_pelajaran,bidang_yang_diampu
0,u002,p051,2021-12-21 00:00:00,2021,3.916520,198505192015031819,B05017869,2851763664130256,141561293502,"Kutsiyah, S.Pd",1984-04-29 00:00:00,IIID,2007-03-13 00:00:00,2012-01-18 00:00:00,Pendidikan Jasmani,Perempuan,Pendidikan Jasmani dan Kesehatan,Guru Pendidikan Jasmani dan Kesehatan
1,u003,p052,2021-12-21 00:00:00,2021,3.916520,198505192015031387,B05017826,2851763664130231,141561293323,"Sendja Indah, S.Pd",1984-01-28 00:00:00,IIID,2006-09-01 00:00:00,2012-11-05 00:00:00,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas I A dan B
2,u005,p053,2021-12-21 00:00:00,2021,3.916520,198505192015032861,B05017963,2851763664130246,141561293388,"Lestari, S.Pd",1987-08-12 00:00:00,IIIC,2011-05-07 00:00:00,2013-11-24 00:00:00,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas III A
3,u006,p054,2021-12-21 00:00:00,2021,3.916520,198505192015031707,B05017874,2851763664130092,141561293455,Dra. Eny Masruroh,1990-10-06 00:00:00,IIIC,2012-09-12 00:00:00,2014-06-12 00:00:00,Pendidikan Agama Islam,Perempuan,Pendidikan Agama Islam dan Budi Pekerti,Guru Pendidikan Agama Islam
4,u008,p056,2021-12-21 00:00:00,2021,3.916520,198505192015032196,B05017814,2851763664130048,141561293443,"Lamidi, S.Pd",1992-09-12 00:00:00,IIIB,2016-02-01 00:00:00,2016-05-19 00:00:00,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas VI A
5,u011,p059,2021-12-21 00:00:00,2021,3.916520,198505192015031734,B05017855,2851763664130132,141561293405,"Sarlan, S.Pd",1988-02-21 00:00:00,IIIC,2011-09-02 00:00:00,2016-09-06 00:00:00,Pendidikan Jasmani,Laki-laki,Pendidikan Jasmani dan Kesehatan,Guru Pendidikan Jasmani dan Kesehatan
6,u012,p060,2021-12-21 00:00:00,2021,3.916520,198505192015031242,B05017848,2851763664130046,141561293590,Dianto,1992-05-15 00:00:00,IIIB,2014-10-17 00:00:00,2016-11-17 00:00:00,Pendidikan Guru Sekolah Dasar,Laki-laki,Guru Kelas,Guru Kelas III B
7,u014,p062,2021-12-21 00:00:00,2021,3.916520,198505192015032533,B05017962,2851763664130168,141561293440,"Dyna Fitriyah, S.Pd",1992-10-20 00:00:00,IIIB,2014-11-19 00:00:00,2017-06-16 00:00:00,Pendidikan Guru Sekolah Dasar,Perempuan,Guru Kelas,Guru Kelas V A
8,u015,p063,2021-12-21 00:00:00,2021,3.916520,198505192015032615,B05017831,2851763664130183,141561293465,"Lily Santoso, S.Pd",1985-06-05 00:00:00,IIID,2008-12-21 00:00:00,2017-06-30 00:00:00,Pendidikan Bahasa Inggris,Perempuan,Bahasa Inggris,Guru Bahasa Inggris
9,u016,p064,2021-12-21 00:00:00,2021,3.916520,198505192015032177,B05017974,2851763664130060,141561293427,"Deni Triandi Tiningtyas, S.Psi",1992-10-11 00:00:00,IIIB,2015-09-19 00:00:00,2017-10-06 00:00:00,Pendidikan Guru Sekolah Dasar,Laki-laki,Guru Pendamping Kelas,Guru Pendamping Kelas Rendah
